# 代码说明

## 环境配置
项目使用到的镜像为：`就不提交，就是玩儿~复现镜像`
```sh
# Env
python==3.8.5
CUDA==11.3

# Package
pytorch==1.11.0
ltp==4.1.5.post2
tqdm==4.62.3
einops==0.4.1
torch==1.11.0
scikit-learn==0.23.2
numpy==1.21.5
```

## 算法

### 属性匹配
#### 整体思路

对于属性匹配任务的主要算法思路为：
1. 对属性赋予唯一的id，并使用`nn.Embedding`对id进行编码处理；
2. 使用软注意力机制生成属性Embedding对于图像特征的注意力，并作用于图像特征上；
3. 将属性Embedding以及通过注意力机制缩放的图像特征拼接后输出神经网络预测该属性与图像是否匹配。

### 方法的创新点
1. 通过id embedding的方式将多分类问题转化为二分类问题，使得任务难度降低；
2. 并且通过软注意力机制对图像特征进行缩放来突出该属性更需要注意到的特征点使得图像特征被更充分的被使用；
3. 通过相等属性替换的方式进行正例增强以及相似属性替换的方式进行负例构造，使模型能够更好的辨识属性与图像的关系；
4. 通过图文匹配则属性全匹配的关系来使图文预测结果指导属性预测结果，进一步提升了模型的预测能力。

#### 网络结构
![](pic/attribute_matching_network.png)

#### 损失函数 
使用`BCEWithLogitsLoss`作为二分类损失函数。

#### 数据扩增
由于只有带有属性匹配标签的数据，而不存在属性不匹配标签的数据，所以需要通过数据扩增的方式产生属性与图片不匹配负例才能合理的进行线下的训练以及验证。我们首先构造了一个属性值关系字典，该字典中对于每个属性值都记录了四个字段：
1. `equal_attr`记录与该属性值相等的属性值；
2. `similar_attr`记录与该属性值同属于一个`key_attr`下的不相等的属性值；
3. `same_category_attr`记录该属性值同大类下其他`key_attr`中的属性，这里的大类分为衣服，裤子，鞋子，包；
4. `unsimilar_attr`则记录该属性值不同大类下其他`key_attr`中的属性

对于每个图片中的每个属性，都有47%的概率保持为正例，其余53%产生负例（使用关系字典中的`similar_attr`进行均匀采样），其中正例中的10%的属性会进行正例增强（使用关系字典中的`equal_attr`进行均匀采样）。

#### 模型集成 
由于模型较小，所以使用10折交叉验证进行模型集成，使用到的coarse数据中图文匹配的89588条数据以及fine的50000条数据，随机将这些数据划分为10份依次取其中的9份进行训练以及1份进行验证。最后将每个模型预测的匹配概率相加取平均来作为最终的匹配置信度，当置信度大于0.5时则认为属性与图文匹配。

#### 算法的其他细节
最后由于这次任务存在图文匹配则属性全匹配的推论，并且官方直接提供了图文负例数据（属性没有）而非我们随机生成的负例，所以图文匹配的训练结果更加符合真实分布，所以我们可以使用图文的预测结果来指导属性的预测结果，这里我们使用了图文匹配则属性全设置为匹配的指导方法使得属性的预测结果。

